In [77]:
# pipenv install pandas numpy matplotlib seaborn plotly scipy scikit-learn

import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px 
import plotly.graph_objects as go 
import numpy as np 
from scipy.stats import chi2_contingency

sns.set_style("whitegrid")

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

### Carregar Dados

In [2]:
df_leads = pd.read_csv(r'.\datasets\leads.csv')

In [3]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Prospect ID                                    9240 non-null   object 
 1   Lead Number                                    9240 non-null   int64  
 2   Lead Origin                                    9240 non-null   object 
 3   Lead Source                                    9204 non-null   object 
 4   Do Not Email                                   9240 non-null   object 
 5   Do Not Call                                    9240 non-null   object 
 6   Converted                                      9240 non-null   int64  
 7   TotalVisits                                    9103 non-null   float64
 8   Total Time Spent on Website                    9240 non-null   int64  
 9   Page Views Per Visit                           9103 

In [4]:
df_leads.head(10)

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified
5,2058ef08-2858-443e-a01f-a9237db2f5ce,660680,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,NaN,NaN,01.High,02.Medium,17.0,15.0,No,No,Modified
6,9fae7df4-169d-489b-afe4-0f3d752542ed,660673,Landing Page Submission,Google,No,No,1,2.0,1640,2.0,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,No,Modified
7,20ef72a2-fb3b-45e0-924e-551c5fa59095,660664,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,NaN,NaN,02.Medium,02.Medium,15.0,15.0,No,No,Modified
8,cfa0128c-a0da-4656-9d47-0aa4e67bf690,660624,Landing Page Submission,Direct Traffic,No,No,0,2.0,71,2.0,...,No,NaN,Thane & Outskirts,02.Medium,02.Medium,14.0,14.0,No,Yes,Email Opened
9,af465dfc-7204-4130-9e05-33231863c4b5,660616,API,Google,No,No,0,4.0,58,4.0,...,No,NaN,Mumbai,02.Medium,02.Medium,13.0,16.0,No,No,Email Opened


### Feature Engineering and Data Cleaning

In [5]:
# Remover colunas Prospect ID e Lead Number
df_leads.drop(columns=['Prospect ID', 'Lead Number'], axis=1, inplace=True)

In [6]:
# Mostrar e remover as colunas categoricas que possuem um valor possivel

for c in df_leads.select_dtypes(include=['object']).columns:
    if df_leads[c].nunique() == 1:
        print(f'Coluna {c} possui um valor possivel: {df_leads[c].unique()}')
        df_leads.drop(columns=[c], axis=1, inplace=True)

Coluna Magazine possui um valor possivel: ['No']
Coluna Receive More Updates About Our Courses possui um valor possivel: ['No']
Coluna Update me on Supply Chain Content possui um valor possivel: ['No']
Coluna Get updates on DM Content possui um valor possivel: ['No']
Coluna I agree to pay the amount through cheque possui um valor possivel: ['No']


In [7]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 30 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Lead Origin                                    9240 non-null   object 
 1   Lead Source                                    9204 non-null   object 
 2   Do Not Email                                   9240 non-null   object 
 3   Do Not Call                                    9240 non-null   object 
 4   Converted                                      9240 non-null   int64  
 5   TotalVisits                                    9103 non-null   float64
 6   Total Time Spent on Website                    9240 non-null   int64  
 7   Page Views Per Visit                           9103 non-null   float64
 8   Last Activity                                  9137 non-null   object 
 9   Country                                        6779 

In [8]:
# Mostrar oss valores possiveis para todas as categoricas
for c in df_leads.select_dtypes(include=['object']).columns:
    print(f'Coluna {c} possui esses vaores possivel: \n{df_leads[c].unique()}\n')

Coluna Lead Origin possui esses vaores possivel: 
['API' 'Landing Page Submission' 'Lead Add Form' 'Lead Import'
 'Quick Add Form']

Coluna Lead Source possui esses vaores possivel: 
['Olark Chat' 'Organic Search' 'Direct Traffic' 'Google' 'Referral Sites'
 'Welingak Website' 'Reference' 'google' 'Facebook' nan 'blog'
 'Pay per Click Ads' 'bing' 'Social Media' 'WeLearn' 'Click2call'
 'Live Chat' 'welearnblog_Home' 'youtubechannel' 'testone' 'Press_Release'
 'NC_EDM']

Coluna Do Not Email possui esses vaores possivel: 
['No' 'Yes']

Coluna Do Not Call possui esses vaores possivel: 
['No' 'Yes']

Coluna Last Activity possui esses vaores possivel: 
['Page Visited on Website' 'Email Opened' 'Unreachable'
 'Converted to Lead' 'Olark Chat Conversation' 'Email Bounced'
 'Email Link Clicked' 'Form Submitted on Website' 'Unsubscribed'
 'Had a Phone Conversation' 'View in browser link Clicked' nan
 'Approached upfront' 'SMS Sent' 'Visited Booth in Tradeshow'
 'Resubscribed to emails' 'Email Rece

In [9]:
# Mostrar o percentual de valores ausentes ou com o valor == 'Select' para cada coluna categorica
for c in df_leads.select_dtypes(include=['object']).columns:
    contagem_nulas = (df_leads[c] == 'Select').sum() + df_leads[c].isnull().sum()
    print(f'Coluna {c} possui essa quantidade de valores ausentes: \n{contagem_nulas / len(df_leads) * 100:.2f}%\n')

Coluna Lead Origin possui essa quantidade de valores ausentes: 
0.00%

Coluna Lead Source possui essa quantidade de valores ausentes: 
0.39%

Coluna Do Not Email possui essa quantidade de valores ausentes: 
0.00%

Coluna Do Not Call possui essa quantidade de valores ausentes: 
0.00%

Coluna Last Activity possui essa quantidade de valores ausentes: 
1.11%

Coluna Country possui essa quantidade de valores ausentes: 
26.63%

Coluna Specialization possui essa quantidade de valores ausentes: 
36.58%

Coluna How did you hear about X Education possui essa quantidade de valores ausentes: 
78.46%

Coluna What is your current occupation possui essa quantidade de valores ausentes: 
29.11%

Coluna What matters most to you in choosing a course possui essa quantidade de valores ausentes: 
29.32%

Coluna Search possui essa quantidade de valores ausentes: 
0.00%

Coluna Newspaper Article possui essa quantidade de valores ausentes: 
0.00%

Coluna X Education Forums possui essa quantidade de valores aus

In [10]:
# Remover colunas categoricas cujo percentual ultrapassou a limiar de 25%
for c in df_leads.select_dtypes(include=['object']).columns:
    contagem_nulas = (df_leads[c] == 'Select').sum() + df_leads[c].isnull().sum()
    if ((contagem_nulas / len(df_leads) * 100) > 25.0):
        print(f'Coluna {c} possui essa quantidade de valores ausentes: \n{contagem_nulas / len(df_leads) * 100:.2f}%\n')
        df_leads.drop(columns=[c], axis=1, inplace=True)

Coluna Country possui essa quantidade de valores ausentes: 
26.63%

Coluna Specialization possui essa quantidade de valores ausentes: 
36.58%

Coluna How did you hear about X Education possui essa quantidade de valores ausentes: 
78.46%

Coluna What is your current occupation possui essa quantidade de valores ausentes: 
29.11%

Coluna What matters most to you in choosing a course possui essa quantidade de valores ausentes: 
29.32%

Coluna Tags possui essa quantidade de valores ausentes: 
36.29%

Coluna Lead Quality possui essa quantidade de valores ausentes: 
51.59%

Coluna Lead Profile possui essa quantidade de valores ausentes: 
74.19%

Coluna City possui essa quantidade de valores ausentes: 
39.71%

Coluna Asymmetrique Activity Index possui essa quantidade de valores ausentes: 
45.65%

Coluna Asymmetrique Profile Index possui essa quantidade de valores ausentes: 
45.65%



In [11]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 19 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Lead Origin                             9240 non-null   object 
 1   Lead Source                             9204 non-null   object 
 2   Do Not Email                            9240 non-null   object 
 3   Do Not Call                             9240 non-null   object 
 4   Converted                               9240 non-null   int64  
 5   TotalVisits                             9103 non-null   float64
 6   Total Time Spent on Website             9240 non-null   int64  
 7   Page Views Per Visit                    9103 non-null   float64
 8   Last Activity                           9137 non-null   object 
 9   Search                                  9240 non-null   object 
 10  Newspaper Article                       9240 non-null   obje

In [12]:
# Na coluna Lead Source, substituir o valor google pelo Google
df_leads['Lead Source'] = df_leads['Lead Source'].apply(lambda x: 'Google' if x =='google' else x)

In [13]:
# Converter valores de colunas categoriicas de yes/no para 1/0:
for c in df_leads.select_dtypes(include=['object']).columns:
    valores_unicos = df_leads[c].unique()
    if set(valores_unicos).issubset(set(['Yes', 'No'])):
        print(f'{c}')
        df_leads[c] = df_leads[c].apply(lambda x: 1 if x == 'Yes' else 0 )

Do Not Email
Do Not Call
Search
Newspaper Article
X Education Forums
Newspaper
Digital Advertisement
Through Recommendations
A free copy of Mastering The Interview


In [14]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 19 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Lead Origin                             9240 non-null   object 
 1   Lead Source                             9204 non-null   object 
 2   Do Not Email                            9240 non-null   int64  
 3   Do Not Call                             9240 non-null   int64  
 4   Converted                               9240 non-null   int64  
 5   TotalVisits                             9103 non-null   float64
 6   Total Time Spent on Website             9240 non-null   int64  
 7   Page Views Per Visit                    9103 non-null   float64
 8   Last Activity                           9137 non-null   object 
 9   Search                                  9240 non-null   int64  
 10  Newspaper Article                       9240 non-null   int6

In [15]:
# Remover linhas com valores ausentes em colunas categoricas
colunas_categoricas = df_leads.select_dtypes(include=['object']).columns
df_leads.dropna(subset=colunas_categoricas, inplace=True)

In [16]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9103 entries, 0 to 9239
Data columns (total 19 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Lead Origin                             9103 non-null   object 
 1   Lead Source                             9103 non-null   object 
 2   Do Not Email                            9103 non-null   int64  
 3   Do Not Call                             9103 non-null   int64  
 4   Converted                               9103 non-null   int64  
 5   TotalVisits                             9074 non-null   float64
 6   Total Time Spent on Website             9103 non-null   int64  
 7   Page Views Per Visit                    9074 non-null   float64
 8   Last Activity                           9103 non-null   object 
 9   Search                                  9103 non-null   int64  
 10  Newspaper Article                       9103 non-null   int64  
 

In [17]:
# Apresentar estatisticas descritivas
df_leads.describe()

,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Search,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Asymmetrique Activity Score,Asymmetrique Profile Score,A free copy of Mastering The Interview
count,9103.000000,9103.000000,9103.000000,9074.000000,9103.000000,9074.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,9103.000000,4944.000000,4944.000000,9103.000000
mean,0.079205,0.000220,0.379216,3.456028,483.773921,2.370151,0.001538,0.000220,0.000110,0.000110,0.000439,0.000769,14.313511,16.340210,0.317258
std,0.270073,0.014822,0.485219,4.858802,545.519186,2.160871,0.039189,0.014822,0.010481,0.010481,0.020959,0.027721,1.394627,1.807428,0.465434
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,11.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,12.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,15.000000,0.000000
50%,0.000000,0.000000,0.000000,3.000000,247.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,16.000000,0.000000
75%,0.000000,0.000000,1.000000,5.000000,924.000000,3.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,18.000000,1.000000
max,1.000000,1.000000,1.000000,251.000000,2272.000000,55.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,18.000000,20.000000,1.000000


In [18]:
# Mostrar e remover as colunas numericas que possuem somente um valor possivel
for c in df_leads.select_dtypes(include=['number']).columns:
    if df_leads[c].nunique() == 1:
        print(f'A coluna {c} possui apenas um valor possivel: {df_leads[c].unique()}')
        df_leads.drop(columns=[c], axis=1, inplace=True)

In [19]:
# Visualizar percentual de valores nulos nas colunas numericas
for c in df_leads.select_dtypes(include=['number']).columns:
    contagem_nulas =  df_leads[c].isnull().sum()
    print(f'Coluna {c} possui essa quantidade de valores ausentes: \n{contagem_nulas / len(df_leads) * 100:.2f}%\n')

Coluna Do Not Email possui essa quantidade de valores ausentes: 
0.00%

Coluna Do Not Call possui essa quantidade de valores ausentes: 
0.00%

Coluna Converted possui essa quantidade de valores ausentes: 
0.00%

Coluna TotalVisits possui essa quantidade de valores ausentes: 
0.32%

Coluna Total Time Spent on Website possui essa quantidade de valores ausentes: 
0.00%

Coluna Page Views Per Visit possui essa quantidade de valores ausentes: 
0.32%

Coluna Search possui essa quantidade de valores ausentes: 
0.00%

Coluna Newspaper Article possui essa quantidade de valores ausentes: 
0.00%

Coluna X Education Forums possui essa quantidade de valores ausentes: 
0.00%

Coluna Newspaper possui essa quantidade de valores ausentes: 
0.00%

Coluna Digital Advertisement possui essa quantidade de valores ausentes: 
0.00%

Coluna Through Recommendations possui essa quantidade de valores ausentes: 
0.00%

Coluna Asymmetrique Activity Score possui essa quantidade de valores ausentes: 
45.69%

Coluna A

In [20]:
# Remover colunas numericas cujo percentual ultrapassou a limiar de 25%
for c in df_leads.select_dtypes(include=['number']).columns:
    contagem_nulas = df_leads[c].isnull().sum()
    if ((contagem_nulas / len(df_leads) * 100) > 25.0):
        print(f'Coluna {c} possui essa quantidade de valores ausentes: \n{contagem_nulas / len(df_leads) * 100:.2f}%\n')
        df_leads.drop(columns=[c], axis=1, inplace=True)

Coluna Asymmetrique Activity Score possui essa quantidade de valores ausentes: 
45.69%

Coluna Asymmetrique Profile Score possui essa quantidade de valores ausentes: 
45.69%



In [21]:
# Remover linhas com valores ausentes em colunas categoricas
colunas_numericas = df_leads.select_dtypes(include=['number']).columns
df_leads.dropna(subset=colunas_numericas, inplace=True)

In [22]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9074 entries, 0 to 9239
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Lead Origin                             9074 non-null   object 
 1   Lead Source                             9074 non-null   object 
 2   Do Not Email                            9074 non-null   int64  
 3   Do Not Call                             9074 non-null   int64  
 4   Converted                               9074 non-null   int64  
 5   TotalVisits                             9074 non-null   float64
 6   Total Time Spent on Website             9074 non-null   int64  
 7   Page Views Per Visit                    9074 non-null   float64
 8   Last Activity                           9074 non-null   object 
 9   Search                                  9074 non-null   int64  
 10  Newspaper Article                       9074 non-null   int64  
 

### EDA

Hit Radio:
- Razão entre leads convertidos em vendas sobe a quantidade total de leads (Conceito 1), 
    ex: 100 leads criados e 30 convertidos = 30%
- Razão entre os leads convertidos em vendas sobe a quantidade de leads encerrados (Conceito 2), 
    x: 100 leads criados, 20 convertidos e 20 não foram convertidos = 50%

In [23]:
# Distribuição da variavel target em percentual

fig = px.bar(df_leads['Converted'].value_counts() / len(df_leads) * 100, 
             title='Hit Radio - Fator de conversão',
             labels={'index': 'Converted', 'value': 'Percentual'},
             opacity=0.8)
fig.update_layout(showlegend=False)
fig.show()

In [25]:
# Matriz de correlação das variáveis numericas com Plotly GO

corr_matrix = df_leads.select_dtypes(include=['number']).corr()

In [36]:
# Plot da matriz de correlação das variáveis numericas com Plotly GO

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = corr_matrix.columns,
        y = corr_matrix.index,
        z = np.array(corr_matrix),
        text = corr_matrix.values,
        texttemplate = '%{text:.2f}',
        colorscale = px.colors.diverging.RdBu,
        zmin = -1,
        zmax = 1
    )
)

In [38]:
# Box Plot Converted x TotalVisits
fig = px.box(df_leads, x='Converted', y='TotalVisits', color='Converted')
fig.show()

In [39]:
# Box Plot Converted x TotalTimeSpent
fig = px.box(df_leads, x='Converted', y='Total Time Spent on Website', color='Converted')
fig.show()

In [41]:
# Box Plot Converted x Page Views Per Visit
fig = px.box(df_leads, x='Converted', y='Page Views Per Visit', color='Converted')
fig.show()

In [42]:
# Criar uma tabela de contingencia de Converted x Lead Source
contingency_table_lead_source = pd.crosstab(df_leads['Converted'], df_leads['Lead Source'])

In [43]:
contingency_table_lead_source

Lead Source,Click2call,Direct Traffic,Facebook,Google,Live Chat,NC_EDM,Olark Chat,Organic Search,Pay per Click Ads,Press_Release,Reference,Referral Sites,Social Media,WeLearn,Welingak Website,bing,blog,testone,welearnblog_Home,youtubechannel
Converted,,,,,,,,,,,,,,,,,,,,
0,1,1725,22,1726,0,0,1305,718,1,2,33,94,1,0,2,5,1,1,1,1
1,3,818,9,1147,2,1,448,436,0,0,410,31,1,1,127,1,0,0,0,0


In [48]:
# Executar o teste de independecia de chi2
chi2, p, dof, expected = chi2_contingency(contingency_table_lead_source)
# Mostrar o resultado
print(f'Estatistica qui-quadrado: {chi2}')
print(f'P-value: {p}')
print(f'Graus de liberdade: {dof}')
print(f'Existe uma relação significativa entre converted e Lead Source? {p < 0.05}')

Estatistica qui-quadrado: 942.1372507753774
P-value: 1.1748671316223743e-187
Graus de liberdade: 19
Existe uma relação significativa entre converted e Lead Source? True


In [49]:
# Criar uma tabela de contingencia de Converted x Lead Origin
contingency_table_lead_origin = pd.crosstab(df_leads['Converted'], df_leads['Lead Origin'])

In [50]:
contingency_table_lead_origin

Lead Origin,API,Landing Page Submission,Lead Add Form,Lead Import
Converted,,,,
0,2463,3118,37,21
1,1115,1767,544,9


In [51]:
# Executar o teste de independecia de chi2
chi2, p, dof, expected = chi2_contingency(contingency_table_lead_origin)
# Mostrar o resultado
print(f'Estatistica qui-quadrado: {chi2}')
print(f'P-value: {p}')
print(f'Graus de liberdade: {dof}')
print(f'Existe uma relação significativa entre converted e Lead Origin? {p < 0.05}')

Estatistica qui-quadrado: 843.1212236836468
P-value: 1.9228780932726904e-182
Graus de liberdade: 3
Existe uma relação significativa entre converted e Lead Origin? True


In [52]:
# Criar uma tabela de contingencia de Converted x Last Notable Activity
contingency_table_lna = pd.crosstab(df_leads['Converted'], df_leads['Last Notable Activity'])

In [53]:
contingency_table_lna

Last Notable Activity,Approached upfront,Email Bounced,Email Link Clicked,Email Marked Spam,Email Opened,Email Received,Form Submitted on Website,Had a Phone Conversation,Modified,Olark Chat Conversation,Page Visited on Website,Resubscribed to emails,SMS Sent,Unreachable,Unsubscribed,View in browser link Clicked
Converted,,,,,,,,,,,,,,,,
0,0,51,128,0,1781,0,1,1,2587,158,225,0,663,10,33,1
1,1,9,45,2,1042,1,0,13,680,25,93,1,1489,22,12,0


In [54]:
# Executar o teste de independecia de chi2
chi2, p, dof, expected = chi2_contingency(contingency_table_lna)
# Mostrar o resultado
print(f'Estatistica qui-quadrado: {chi2}')
print(f'P-value: {p}')
print(f'Graus de liberdade: {dof}')
print(f'Existe uma relação significativa entre converted e Last Notable Activity? {p < 0.05}')

Estatistica qui-quadrado: 1424.6171966295433
P-value: 8.365508263958168e-295
Graus de liberdade: 15
Existe uma relação significativa entre converted e Last Notable Activity? True


### Preparação dos dados

In [72]:
X = df_leads.drop(columns=['Converted'])
y = df_leads['Converted']

In [73]:
# Criar um Colunm Transformer
colunas_numericas = X.select_dtypes(include=['number']).columns
colunas_categoricas = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers= [
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(handle_unknown='ignore'), colunas_categoricas)
    ]   
)

In [74]:
# Dividr conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=51)

# Aplicar Colunm Transoformer
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [76]:
print(X_train.shape)
print(X_test.shape)

(7259, 68)
(1815, 68)


### Treinamento do modelo

In [153]:
# Criar modelo de BaggingClassifier

bagging_model = BaggingClassifier(
    estimator=LogisticRegression(),
    n_estimators=5,
    random_state=51
)

In [139]:
# Criar modelo de BaggingClassifier (Some more things)

bagging_model = BaggingClassifier(
    estimator=LogisticRegression(),
    n_estimators=10,
    random_state=51,
    max_samples=0.3,  #quantidade de registros em cada sample
    max_features=0.3  #quantidade de features por sample
)

In [154]:
# Treinar modelo
bagging_model.fit(X_train, y_train)

,estimator,LogisticRegression()
,n_estimators,5
,max_samples,1.0
,max_features,1.0
,bootstrap,True
,bootstrap_features,False
,oob_score,False
,warm_start,False
,n_jobs,None
,random_state,51
,verbose,0


### Avaliação do modelo

In [155]:
# Fazer predições no conjunto de testes
y_pred = bagging_model.predict(X_test)

In [156]:
# Avaliar modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall= recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [157]:
# Mostrar Resultados
print(f'Acuracia: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')

Acuracia: 0.7988980716253443
Precision: 0.747163695299838
Recall: 0.6880597014925374
F1-Score: 0.7163947163947164


In [158]:
# Mostr matrix de confusao em Plotly 
conf_matrix = confusion_matrix(y_test, y_pred)
fig = px.imshow(conf_matrix,
                labels=dict(x='Predição', y='Real', color='Contagem'),
                x=['Not Converted', 'Converted'],
                y=['Not Converted', 'Converted'],
                color_continuous_scale='Viridis'
                )

fig.update_traces(text=conf_matrix, texttemplate="%{z}")
fig.update_layout(coloraxis_showscale=False)
fig.show()


In [159]:
# Calcular a importancia das features
importances = np.mean([np.abs(estimator.coef_[0]) for estimator in bagging_model.estimators_], axis = 0)

In [160]:
# Obter o nome das features após o preprocessor
feature_names = (colunas_numericas.tolist() + 
                 preprocessor.named_transformers_['cat']
                 .get_feature_names_out(colunas_categoricas).tolist())

In [161]:
feature_names

['Do Not Email',
 'Do Not Call',
 'TotalVisits',
 'Total Time Spent on Website',
 'Page Views Per Visit',
 'Search',
 'Newspaper Article',
 'X Education Forums',
 'Newspaper',
 'Digital Advertisement',
 'Through Recommendations',
 'A free copy of Mastering The Interview',
 'Lead Origin_API',
 'Lead Origin_Landing Page Submission',
 'Lead Origin_Lead Add Form',
 'Lead Origin_Lead Import',
 'Lead Source_Click2call',
 'Lead Source_Direct Traffic',
 'Lead Source_Facebook',
 'Lead Source_Google',
 'Lead Source_Live Chat',
 'Lead Source_NC_EDM',
 'Lead Source_Olark Chat',
 'Lead Source_Organic Search',
 'Lead Source_Pay per Click Ads',
 'Lead Source_Reference',
 'Lead Source_Referral Sites',
 'Lead Source_Social Media',
 'Lead Source_WeLearn',
 'Lead Source_Welingak Website',
 'Lead Source_bing',
 'Lead Source_blog',
 'Lead Source_testone',
 'Lead Source_welearnblog_Home',
 'Lead Source_youtubechannel',
 'Last Activity_Approached upfront',
 'Last Activity_Converted to Lead',
 'Last Activity_

In [162]:
# Criar df combinando feature_name e importances 
df_feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

In [163]:
# Ordenar pela importancia
df_feature_importances = df_feature_importances.sort_values(by='Importance', ascending=True)

In [164]:
# Plotar a importancia das features
fig = px.bar(df_feature_importances, x='Importance', y='Feature', orientation='h', title='Importância das features (Com base nos coeficientes absolutos)')
fig.update_layout(height=1280, width=1000, yaxis={'categoryorder': 'total ascending'})
fig.show()

### Some More Things

In [149]:
bagging_model.estimators_samples_

[array([6406,  685, 6836, ..., 1652, 3092, 1535],
       shape=(2177,), dtype=int32),
 array([4792, 1818,  976, ..., 3085, 2558, 3027],
       shape=(2177,), dtype=int32),
 array([3886, 2327, 5553, ..., 6337, 1302, 5998],
       shape=(2177,), dtype=int32),
 array([6393, 6750, 2647, ..., 3684, 4862, 7077],
       shape=(2177,), dtype=int32),
 array([4484, 5292, 5462, ..., 3761, 5032, 5102],
       shape=(2177,), dtype=int32),
 array([3265, 1934, 1857, ..., 2656, 3488, 7207],
       shape=(2177,), dtype=int32),
 array([4257, 1116, 3993, ..., 5225, 5510, 4599],
       shape=(2177,), dtype=int32),
 array([6792, 1181,  504, ..., 2153, 5534, 3245],
       shape=(2177,), dtype=int32),
 array([ 746,  436, 4976, ...,  508, 2021,   33],
       shape=(2177,), dtype=int32),
 array([1008, 1458, 1182, ..., 2082, 4120, 1192],
       shape=(2177,), dtype=int32)]

In [150]:
bagging_model.estimators_features_

[array([ 5, 12,  7,  2, 46, 40, 45,  9,  8, 56,  4, 54, 32, 63, 53, 34, 51,
        33, 59, 50], dtype=int32),
 array([13, 19, 63, 34, 43, 30, 67, 29, 55, 21,  7, 52, 47, 59, 14, 56, 66,
        57, 39, 27], dtype=int32),
 array([10,  3,  5, 52, 20, 41, 63, 17, 37,  2, 44, 45,  9, 32, 57, 48, 40,
        24, 25, 16], dtype=int32),
 array([11, 24, 51, 48,  6, 64, 34, 60, 45, 25, 63, 67, 17, 27, 14, 56, 57,
         5, 41, 43], dtype=int32),
 array([13, 19, 59, 62, 50,  4,  7, 23, 47, 64, 21,  9, 12, 29, 35, 52,  2,
        45, 28, 51], dtype=int32),
 array([58, 25, 46, 67, 20, 32, 36, 37, 64, 21, 39, 27, 11, 22,  1, 53,  7,
        24, 55, 66], dtype=int32),
 array([30, 56, 61, 19, 28, 43, 11, 18, 45, 46, 36, 16, 54,  3, 41, 17, 63,
        29, 12,  9], dtype=int32),
 array([25, 48, 37, 61, 30, 47, 29, 13, 46, 64, 43, 53, 17,  8, 32, 27, 63,
        18, 66, 24], dtype=int32),
 array([67, 47,  3,  2, 21, 52, 64, 55, 42,  0, 38, 33,  5, 41, 10,  6,  1,
        35, 62,  4], dtype=int32),
 

In [151]:
# Predizer Probabilidade de Conversão
y_pred_prob = bagging_model.predict_proba(X_test)

In [152]:
y_pred_prob

array([[0.40769847, 0.59230153],
       [0.74805414, 0.25194586],
       [0.74805414, 0.25194586],
       ...,
       [0.78117086, 0.21882914],
       [0.81479581, 0.18520419],
       [0.27125281, 0.72874719]], shape=(1815, 2))

### Cenário de CRM - Utilidade de Probabilidade

- CRM
    Leads concluídos - Resultado Positivo ou Negativo
    Leads em aberto - Não tenho resultado

Treino um modelo no que esta cncluido para que ele generalize bem no que esta aberto

Lead em aberto
- Proababilidade de converter
  Quando muito alta podemos olhar com mais foco para realmente converter
  Quando muito baixo podemos meio que descartar
+ Importancia das features 

### Salvar Dados e Preprocessor do modelo

In [165]:
import joblib

df_leads.to_csv('.\datasets\leads_cleaned.csv', index=False)
joblib.dump(preprocessor, '.\preprocessor__dataset_leads.pkl')

['.\\preprocessor__dataset_leads.pkl']